In [ ]:
import pandas as pd

cities_df = pd.read_excel('/content/ПФО РБ.xls', sheet_name = 'Лист1')
distances_df = pd.read_excel('/content/ПФО РБ.xls', sheet_name = 'Расстояния')

In [ ]:
distances_df

,Unnamed: 0,A,Барановичи,"Берёза, Берёзовский район",Бобруйск,"Борисов, Минская обл.",Брест,Витебск,"Глубокое, Глубокский район",Гомель,...,"Орша, Витебская обл.",Пинск,"Речица, Речицкий район","Светлогорск, Светлогорский район",Слоним,Слуцк,Сморгонь,Солигорск,Фаниполь,END
0,A,0,150,245,143,77,350,269,167,310,...,220,285,275,220,200,105,120,140,30,1000000
1,Барановичи,150,0,108,240,230,214,430,320,420,...,360,167,360,310,60,114,190,127,121,490
2,"Берёза, Берёзовский район",245,108,0,330,330,110,530,410,470,...,460,112,430,370,93,192,288,205,220,590
3,Бобруйск,143,240,330,0,147,410,278,280,159,...,196,300,133,77,300,116,264,133,165,292
4,"Борисов, Минская обл.",77,230,330,147,0,440,207,135,310,...,137,380,280,224,281,181,165,210,117,253
5,Брест,350,214,111,410,440,0,630,520,530,...,560,180,490,470,170,300,390,310,330,680
6,Витебск,269,430,530,278,207,630,0,183,330,...,81,560,370,330,460,360,300,400,300,135
7,"Глубокое, Глубокский район",167,320,410,280,135,520,183,0,450,...,230,460,410,360,350,276,144,310,205,340
8,Гомель,310,420,470,159,310,530,330,450,0,...,250,360,55,116,480,280,430,294,330,350
9,Гродно,277,200,190,430,370,240,530,370,590,...,500,270,550,490,135,300,237,310,275,610


In [ ]:
!pip install deap

import numpy as np
import random
import pandas as pd
from deap import base, creator, tools, algorithms

def load_data_from_excel(file_path):
    """
    Загружает данные из Excel файла с двумя листами
    """
    # Загружаем данные о городах
    cities_df = pd.read_excel(file_path, sheet_name='Лист1')

    # Загружаем матрицу расстояний
    distances_df = pd.read_excel(file_path, sheet_name='Расстояния', index_col=0)

    # Создаем словарь data
    data = {}
    for _, row in cities_df.iterrows():
        city = row['Город']
        data[city] = {
            'max_cubic': row['max_cubic'],
            'volume_to_route_point': row['volume_to_route_point'],
            'volume_to_carry': row['volume_to_carry']
        }

    # Получаем список всех городов
    VERTICES = cities_df['Город'].tolist()

    # Убедимся, что матрица расстояний имеет правильный порядок
    # Приведем названия городов к одинаковому формату
    distances_df.index = distances_df.index.astype(str).str.strip()
    distances_df.columns = distances_df.columns.astype(str).str.strip()

    # Убедимся, что все города из VERTICES есть в матрице расстояний
    missing_in_index = set(VERTICES) - set(distances_df.index)
    missing_in_columns = set(VERTICES) - set(distances_df.columns)

    if missing_in_index:
        print(f"Города отсутствуют в индексах матрицы расстояний: {missing_in_index}")

    if missing_in_columns:
        print(f"Города отсутствуют в колонках матрицы расстояний: {missing_in_columns}")

    # Переиндексируем матрицу расстояний
    distances_df = distances_df.reindex(index=VERTICES, columns=VERTICES)

    # Заполним пропущенные значения большими числами (штраф за отсутствие связи)
    distances_df = distances_df.fillna(10000)

    ROUTESKM = distances_df.values

    return data, ROUTESKM, VERTICES

# Загрузка данных из файла
file_path = 'ПФО РБ.xls'
data, ROUTESKM, VERTICES = load_data_from_excel(file_path)

available_cubics = [25, 40, 50, 82]

# Создаем mapping для кодирования вершин в числа
vertex_to_code = {v: i for i, v in enumerate(VERTICES)}
code_to_vertex = {i: v for i, v in enumerate(VERTICES)}

# Промежуточные вершины (все, кроме A и END)
intermediate_vertices = [v for v in VERTICES if v not in ['A', 'END']]
intermediate_codes = [vertex_to_code[v] for v in intermediate_vertices]
end_code = vertex_to_code['END']
start_code = vertex_to_code['A']

# Параметры генетического алгоритма
POPULATION_SIZE = 200  # Увеличиваем для большего разнообразия
P_CROSSOVER = 0.9
P_MUTATION = 0.4  # Увеличиваем для большего разнообразия
MAX_GENERATIONS = 100000  # Увеличиваем для лучшей сходимости
HOF_SIZE = 10

# Удаляем предыдущие определения классов, чтобы избежать предупреждений
if hasattr(creator, 'FitnessMin'):
    del creator.FitnessMin
if hasattr(creator, 'Individual'):
    del creator.Individual

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Функции для вычисления параметров маршрута
def decode_route(route_codes):
    """Декодирует маршрут из кодов в названия вершин"""
    return [code_to_vertex[code] for code in route_codes]

def calculate_route_volume(route):
    """Вычисляет объем груза в каждой точке маршрута"""
    route = decode_route(route)
    volumes = []
    current_volume = data[route[0]]['volume_to_carry']

    for i in range(1, len(route)):
        if route[i] != 'END':
            current_volume += data[route[i]]['volume_to_route_point']

    volumes.append(current_volume)

    for i in range(1, len(route)-1):
        current_volume = current_volume - data[route[i]]['volume_to_route_point'] + data[route[i]]['volume_to_carry']
        volumes.append(current_volume)

    return volumes

def calculate_route_distance(route):
    """Вычисляет расстояние маршрута"""
    route = decode_route(route)
    distance = 0
    for i in range(len(route)-1):
        from_idx = VERTICES.index(route[i])
        to_idx = VERTICES.index(route[i+1])
        distance += ROUTESKM[from_idx][to_idx]
    return distance

def get_min_max_cubic(route):
    """Возвращает минимальную максимальную кубатуру на маршруте"""
    route = decode_route(route)
    return min(data[vertex]['max_cubic'] for vertex in route)

# Функция оценки приспособленности
def eval_routes(individual):
    """Оценивает приспособленность особи с использованием сбалансированной метрики"""
    total_balanced_cost = 0
    penalty = 0

    # Преобразуем особь в список маршрутов
    routes = []
    current_route = [start_code]

    for gene in individual:
        if gene == end_code:
            current_route.append(end_code)
            routes.append(current_route)
            current_route = [start_code]
        else:
            current_route.append(gene)

    if len(current_route) > 1:
        current_route.append(end_code)
        routes.append(current_route)

    # Проверяем, что все промежуточные вершины посещены ровно один раз
    visited_vertices = set()
    duplicate_vertices = set()

    # Проверяем каждый маршрут на повторения внутри него
    for route in routes:
        route_vertices = set()
        for vertex_code in route:
            if vertex_code in intermediate_codes:
                if vertex_code in route_vertices:
                    # Штраф за повторение вершины внутри одного маршрута
                    penalty += 50000
                    duplicate_vertices.add(vertex_code)
                else:
                    route_vertices.add(vertex_code)

                if vertex_code in visited_vertices:
                    # Штраф за повторение вершины в разных маршрутах
                    penalty += 1000000
                    duplicate_vertices.add(vertex_code)
                else:
                    visited_vertices.add(vertex_code)

    # Штраф за непосещенные вершины
    if visited_vertices != set(intermediate_codes):
        penalty += 100000 * len(set(intermediate_codes) - visited_vertices)

    # Штраф за дубликаты вершин (чем больше дубликатов, тем больше штраф)
    penalty += 200000 * len(duplicate_vertices)

    # Оцениваем каждый маршрут с использованием сбалансированной метрики
    for route in routes:
        # Проверяем, что маршрут содержит хотя бы одну промежуточную вершину
        if len(route) <= 2:
            penalty += 10000
            continue

        try:
            # Вычисляем параметры маршрута
            volumes = calculate_route_volume(route)
            max_volume = max(volumes)
            min_max_cubic = get_min_max_cubic(route)
            distance = calculate_route_distance(route)

            # Проверяем ограничения по кубатуре
            if max_volume > min_max_cubic:
                penalty += 10000 * (max_volume - min_max_cubic)
                # Для невалидных маршрутов используем стандартную метрику
                total_balanced_cost += distance * 1000  # Большой множитель для невалидных маршрутов
            else:
                # Выбираем подходящую кубатуру машины
                suitable_cubics = [c for c in available_cubics if c >= max_volume and c <= min_max_cubic]
                if not suitable_cubics:
                    penalty += 10000
                    total_balanced_cost += distance * 1000  # Большой множитель для невалидных маршрутов
                else:
                    # Минимизируем разницу между кубатурой и максимальной загрузкой
                    best_cubic = min(suitable_cubics, key=lambda x: x - max_volume)

                    # Вычисляем коэффициент утилизации
                    utilization_ratio = max_volume / best_cubic

                    # Вычисляем сбалансированную стоимость маршрута
                    # ((1 - utilization_ratio) + 0.01) * distance * 500
                    balanced_route_cost = distance * 200

                    total_balanced_cost += balanced_route_cost

        except Exception as e:
            # Если возникла ошибка при вычислениях, добавляем большой штраф
            penalty += 100000
            print(f"Ошибка при вычислении маршрута: {e}")

    return total_balanced_cost + penalty,

# Создаем инструменты генетического алгоритма
toolbox = base.Toolbox()

# Генератор случайных генов
def random_gene():
    return random.choice(intermediate_codes + [end_code])

# Регистрируем функции
toolbox.register("gene", random_gene)
# Длина хромосомы должна быть достаточной для покрытия всех вершин
# Умножаем на 2, чтобы учесть маркеры конца маршрута
chromosome_length = len(intermediate_codes) * 2
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.gene, n=chromosome_length)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", eval_routes)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=min(intermediate_codes + [end_code]),
                 up=max(intermediate_codes + [end_code]), indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)



import numpy as np
from copy import deepcopy

def apply_2opt_to_best_solutions(hof, data, vertices, ROUTESKM, available_cubics):
    """
    Применяет алгоритм 2-opt к лучшим решениям из Hall of Fame с учетом ограничений по кубатуре
    """
    optimized_solutions = []

    for individual in hof:
        # Декодируем особь в список маршрутов
        routes = []
        current_route = ['A']

        for gene in individual:
            city = code_to_vertex[gene]
            if city == 'END':
                current_route.append('END')
                routes.append(current_route)
                current_route = ['A']
            else:
                current_route.append(city)

        if len(current_route) > 1:
            current_route.append('END')
            routes.append(current_route)

        # Оптимизируем каждый маршрут с помощью 2-opt
        optimized_routes = []
        for route in routes:
            if len(route) > 3:  # Оптимизируем только маршруты с достаточным количеством городов
                optimized_route = optimize_route_2opt(route, data, vertices, ROUTESKM, available_cubics)
                optimized_routes.append(optimized_route)
            else:
                optimized_routes.append(route)

        # Создаем новую особь с оптимизированными маршрутами
        new_individual = []
        for route in optimized_routes:
            for i, city in enumerate(route):
                if i == 0:  # Пропускаем начальный 'A'
                    continue
                new_individual.append(vertex_to_code[city])

        # Создаем новую особь и вычисляем ее приспособленность
        new_ind = creator.Individual(new_individual)
        new_ind.fitness.values = toolbox.evaluate(new_ind)
        optimized_solutions.append(new_ind)

    return optimized_solutions

def optimize_route_2opt(route, data, vertices, ROUTESKM, available_cubics):
    """
    Оптимизирует маршрут с помощью алгоритма 2-opt с учетом ограничений по кубатуре
    """
    # Получаем текущие параметры маршрута
    current_volumes = calculate_route_volume_by_names(route)
    current_max_volume = max(current_volumes)
    current_min_max_cubic = min(data[city]['max_cubic'] for city in route)
    current_distance = calculate_route_distance_by_names(route)

    # Определяем текущую кубатуру машины
    suitable_cubics = [c for c in available_cubics if c >= current_max_volume and c <= current_min_max_cubic]
    if not suitable_cubics:
        return route  # Невозможно оптимизировать - нет подходящей машины

    current_cubic = min(suitable_cubics, key=lambda x: x - current_max_volume)

    # Применяем алгоритм 2-opt
    improved = True
    best_route = route[:]
    best_distance = current_distance
    best_max_volume = current_max_volume
    best_cubic = current_cubic

    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route) - 1):
                if j - i == 1:
                    continue

                # Создаем новый маршрут с переставленными ребрами
                new_route = best_route[:]
                new_route[i:j+1] = best_route[j:i-1:-1]

                # Проверяем ограничения по кубатуре
                new_volumes = calculate_route_volume_by_names(new_route)
                new_max_volume = max(new_volumes)
                new_min_max_cubic = min(data[city]['max_cubic'] for city in new_route)

                # Проверяем, можно ли использовать текущую машину или нужна другая
                if new_max_volume > new_min_max_cubic:
                    continue  # Недопустимый маршрут - превышено ограничение какой-то точки

                # Выбираем подходящую кубатуру машины
                new_suitable_cubics = [c for c in available_cubics if c >= new_max_volume and c <= new_min_max_cubic]
                if not new_suitable_cubics:
                    continue  # Нет подходящей машины

                new_cubic = min(new_suitable_cubics, key=lambda x: x - new_max_volume)

                # Если это предельная кубатура для маршрута и она изменилась, отменяем перестановку
                if new_cubic != best_cubic and new_cubic == new_min_max_cubic:
                    continue

                # Вычисляем новое расстояние
                new_distance = calculate_route_distance_by_names(new_route)

                # Если маршрут лучше, принимаем его
                if new_distance < best_distance:
                    best_route = new_route
                    best_distance = new_distance
                    best_max_volume = new_max_volume
                    best_cubic = new_cubic
                    improved = True

    return best_route

def calculate_route_volume_by_names(route_names):
    """
    Вычисляет объем груза в каждой точке маршрута (по названиям городов)
    """
    volumes = []
    current_volume = data[route_names[0]]['volume_to_carry']

    for i in range(1, len(route_names)):
        if route_names[i] != 'END':
            current_volume += data[route_names[i]]['volume_to_route_point']

    volumes.append(current_volume)

    for i in range(1, len(route_names)-1):
        current_volume = current_volume - data[route_names[i]]['volume_to_route_point'] + data[route_names[i]]['volume_to_carry']
        volumes.append(current_volume)

    return volumes

def calculate_route_distance_by_names(route_names):
    """
    Вычисляет расстояние маршрута (по названиям городов)
    """
    distance = 0
    for i in range(len(route_names)-1):
        from_idx = VERTICES.index(route_names[i])
        to_idx = VERTICES.index(route_names[i+1])
        distance += ROUTESKM[from_idx][to_idx]
    return distance

# Модифицируем основную функцию для применения 2-opt к лучшим решениям
if __name__ == "__main__":
    population, logbook, hof = main()

    # Применяем 2-opt к лучшим решениям
    optimized_hof = apply_2opt_to_best_solutions(hof, data, VERTICES, ROUTESKM, available_cubics)

    # Выводим оптимизированные решения
    print("Оптимизированные решения (2-opt):")
    for i, individual in enumerate(optimized_hof):
        print(f"Решение {i+1}:")

        # Декодируем маршруты
        routes = []
        current_route = ['A']

        for gene in individual:
            vertex = code_to_vertex[gene]
            if vertex == 'END':
                current_route.append('END')
                routes.append(current_route)
                current_route = ['A']
            else:
                current_route.append(vertex)

        if len(current_route) > 1:
            current_route.append('END')
            routes.append(current_route)

        # Вычисляем параметры для каждого маршрута
        total_distance = 0
        for route in routes:
            # Пропускаем маршруты, которые только A->END
            if len(route) <= 2:
                continue

            volumes = calculate_route_volume_by_names(route)
            max_volume = max(volumes)
            min_max_cubic = min(data[v]['max_cubic'] for v in route)
            distance = calculate_route_distance_by_names(route)

            # Выбираем подходящую кубатуру машины
            suitable_cubics = [c for c in available_cubics if c >= max_volume and c <= min_max_cubic]
            best_cubic = min(suitable_cubics, key=lambda x: x - max_volume) if suitable_cubics else "НЕТ"

            print(f"Маршрут: {route}, Кубатура: {best_cubic}, Макс. объем: {max_volume:.2f}, Расстояние: {distance}")
            total_distance += distance

        print(f"Общая приспособленность: {individual.fitness.values[0]}, Общее расстояние: {total_distance}")
        print()

Streaming output truncated to the last 5000 lines.
95112	185   	4.8323e+06 	1.6264e+06 	1.93406e+07
95113	187   	4.90103e+06	1.6264e+06 	1.82032e+07
95114	190   	4.81666e+06	1.6264e+06 	2.20097e+07
95115	191   	5.26819e+06	1.6264e+06 	1.98677e+07
95116	196   	5.65913e+06	1.6264e+06 	1.75846e+07
95117	186   	5.39525e+06	1.6264e+06 	1.95608e+07
95118	189   	5.35403e+06	1.6264e+06 	2.07125e+07
95119	192   	5.14241e+06	1.6264e+06 	1.69798e+07
95120	184   	5.85271e+06	1.6264e+06 	1.76945e+07
95121	190   	5.56799e+06	1.6264e+06 	1.73529e+07
95122	193   	5.4715e+06 	1.6264e+06 	2.0738e+07 
95123	190   	4.84289e+06	1.6264e+06 	1.76501e+07
95124	190   	5.81861e+06	1.6264e+06 	1.59713e+07
95125	183   	5.91896e+06	1.6264e+06 	1.97601e+07
95126	188   	5.66246e+06	1.6264e+06 	1.79557e+07
95127	184   	5.24535e+06	1.6264e+06 	1.9517e+07 
95128	185   	5.23868e+06	1.6264e+06 	1.86292e+07
95129	187   	4.99584e+06	1.6264e+06 	1.77118e+07
95130	187   	5.59785e+06	1.6264e+06 	1.99548e+07
95131	191   	4.984

In [ ]:
def solve_tsp(distance_matrix):
    """
    Решает задачу коммивояжера с помощью OR-Tools
    """
    # Создаем менеджер маршрутов
    manager = pywrapcp.RoutingIndexManager(
        len(distance_matrix),  # количество точек
        1,  # количество транспортных средств
        'A'   # начальная точка
    )

    # Создаем модель маршрутизации
    routing = pywrapcp.RoutingModel(manager)

    # Определяем функцию стоимости
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)+300*len(route)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Настраиваем параметры поиска
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Решаем задачу
    solution = routing.SolveWithParameters(search_parameters)

    # Извлекаем решение
    if solution:
        index = routing.Start(0)
        route_nodes = []
        while not routing.IsEnd(index):
            route_nodes.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        return route_nodes

    return list(range(len(distance_matrix)))

In [ ]:
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import itertools

def intergenerational_optimization(hof, coordinates, vertices, data, routeskm, available_cubics, optimization_interval=1000):
    """
    Межпоколенческая оптимизация для лучших решений
    """
    optimized_solutions = []

    for individual in hof:
        # Декодируем особь в список маршрутов
        routes = decode_individual_to_routes(individual)

        # Вычисляем матрицу абсолютных расстояний до магистральных прямых
        abs_dist_matrix = calculate_absolute_distances(coordinates, vertices)

        # Применяем оптимизацию на основе магистральных прямых
        optimized_routes = optimize_with_magistrals(routes, abs_dist_matrix, coordinates, vertices, data, routeskm, available_cubics)

        # Применяем 2-opt к оптимизированным маршрутам
        final_routes = []
        for route in optimized_routes:
            if len(route) > 3:
                optimized_route = apply_2opt_to_route(route, routeskm, vertices)
                final_routes.append(optimized_route)
            else:
                final_routes.append(route)

        # Создаем новую особь
        new_individual = encode_routes_to_individual(final_routes)
        new_ind = creator.Individual(new_individual)
        new_ind.fitness.values = toolbox.evaluate(new_ind)
        optimized_solutions.append(new_ind)

    return optimized_solutions

def calculate_absolute_distances(coordinates, vertices):
    """
    Вычисляет матрицу абсолютных расстояний от точек до магистральных прямых
    """
    n = len(vertices)
    abs_dist_matrix = np.zeros((n, n, n))

    for i in range(n):
        for k in range(n):
            for l in range(n):
                if k == i or l == i or k == l:
                    abs_dist_matrix[i, k, l] = 0
                else:
                    # Вычисляем расстояние от точки i до прямой (k, l)
                    point = coordinates[i]
                    line_p1 = coordinates[k]
                    line_p2 = coordinates[l]

                    # Формула расстояния от точки до прямой в 3D пространстве
                    numerator = np.linalg.norm(np.cross(line_p2 - line_p1, line_p1 - point))
                    denominator = np.linalg.norm(line_p2 - line_p1)

                    if denominator == 0:
                        abs_dist_matrix[i, k, l] = np.linalg.norm(point - line_p1)
                    else:
                        abs_dist_matrix[i, k, l] = numerator / denominator

    return abs_dist_matrix

def optimize_with_magistrals(routes, abs_dist_matrix, coordinates, vertices, data, routeskm, available_cubics, const=50):
    """
    Оптимизирует маршруты на основе магистральных прямых
    """
    optimized_routes = [route[:] for route in routes]

    # Ранжируем маршруты по длине
    route_lengths = [calculate_route_distance(route, vertices, routeskm) for route in optimized_routes]
    sorted_indices = np.argsort(route_lengths)[::-1]

    for i_idx in sorted_indices:
        I = optimized_routes[i_idx]
        if len(I) <= 2:
            continue

        # Пытаемся переместить все точки из I в другие маршруты
        moved_points = set()

        for city in I:
            if city in ['A', 'END']:
                continue

            # Пропускаем уже перемещенные точки
            if city in moved_points:
                continue

            # Для каждого другого маршрута J
            for j_idx in range(len(optimized_routes)):
                if j_idx == i_idx:
                 continue

                J = optimized_routes[j_idx]

                # Проверяем, можно ли включить все точки I в J
                if can_include_all_points(I, J, abs_dist_matrix, vertices, data, available_cubics, const):
                    # Перемещаем все точки из I в J
                    for point in I:
                        if point not in ['A', 'END'] and point not in moved_points:
                            # Находим лучшую позицию для вставки точки в J
                            best_pos = find_best_position_for_point(point, J, abs_dist_matrix, vertices)
                            J.insert(best_pos, point)
                            moved_points.add(point)

                    # Очищаем маршрут I (оставляем только A и END)
                    optimized_routes[i_idx] = ['A', 'END']
                    break

                else:
                    # Пытаемся переместить отдельные точки
                    magistral_list = []
                    magistrals_dict = defaultdict(list)

                    # Создаем список магистральных отрезков для маршрута J
                    for k in range(len(J) - 1):
                        magistral_list.append((J[k], J[k+1]))
                        magistrals_dict[(J[k], J[k+1])] = J

                    # Для каждой точки в I находим ближайшую магистраль
                    point_magistral_distances = []
                    for point in I:
                        if point in ['A', 'END']:
                            continue

                        point_idx = vertices.index(point)
                        min_dist = float('inf')
                        best_magistral = None

                        for magistral in magistral_list:
                            k_idx = vertices.index(magistral[0])
                            l_idx = vertices.index(magistral[1])
                            dist = abs_dist_matrix[point_idx, k_idx, l_idx]

                            if dist < min_dist:
                                min_dist = dist
                                best_magistral = magistral

                        point_magistral_distances.append((point, best_magistral, min_dist))

                    # Сортируем точки по расстоянию до магистрали
                    point_magistral_distances.sort(key=lambda x: x[2])

                    # Пытаемся переместить точки
                    for point, magistral, dist in point_magistral_distances:
                        if dist > const:
                            continue

                        # Проверяем ограничения по объему
                        test_J = J[:]
                        magistral_idx = test_J.index(magistral[0])
                        test_J.insert(magistral_idx + 1, point)

                        if check_route_constraints(test_J, data, available_cubics):
                            # Проверяем, уменьшится ли общее расстояние
                            old_distance_I = calculate_route_distance(I, vertices, routeskm)
                            old_distance_J = calculate_route_distance(J, vertices, routeskm)

                            test_I = [p for p in I if p != point]
                            new_distance_I = calculate_route_distance(test_I, vertices, routeskm)
                            new_distance_J = calculate_route_distance(test_J, vertices, routeskm)

                            if (new_distance_I + new_distance_J) < (old_distance_I + old_distance_J):
                                # Перемещаем точку
                                J.insert(magistral_idx + 1, point)
                                optimized_routes[i_idx] = test_I
                                moved_points.add(point)
                                break

    return optimized_routes

def can_include_all_points(source_route, target_route, abs_dist_matrix, vertices, data, available_cubics, const):
    """
    Проверяет, можно ли включить все точки из source_route в target_route
    """
    # Проверяем ограничения по объему
    test_route = target_route[:]
    for point in source_route:
        if point not in ['A', 'END']:
            test_route.append(point)

    if not check_route_constraints(test_route, data, available_cubics):
        return False

    # Проверяем, что все точки source_route близки к магистралям target_route
    for point in source_route:
        if point in ['A', 'END']:
            continue

        point_idx = vertices.index(point)
        min_dist = float('inf')

        # Находим минимальное расстояние от точки до магистралей target_route
        for k in range(len(target_route) - 1):
            k_idx = vertices.index(target_route[k])
            l_idx = vertices.index(target_route[k+1])
            dist = abs_dist_matrix[point_idx, k_idx, l_idx]

            if dist < min_dist:
                min_dist = dist

        if min_dist > const:
            return False

    return True

def find_best_position_for_point(point, route, abs_dist_matrix, vertices):
    """
    Находит лучшую позицию для вставки точки в маршрут
    """
    point_idx = vertices.index(point)
    best_pos = 1
    min_dist = float('inf')

    for i in range(len(route) - 1):
        k_idx = vertices.index(route[i])
        l_idx = vertices.index(route[i+1])
        dist = abs_dist_matrix[point_idx, k_idx, l_idx]

        if dist < min_dist:
            min_dist = dist
            best_pos = i + 1

    return best_pos

def decode_individual_to_routes(individual):
    """
    Декодирует особь в список маршрутов
    """
    routes = []
    current_route = ['A']

    for gene in individual:
        city = code_to_vertex[gene]
        if city == 'END':
            current_route.append('END')
            routes.append(current_route)
            current_route = ['A']
        else:
            current_route.append(city)

    if len(current_route) > 1:
        current_route.append('END')
        routes.append(current_route)

    return routes

def encode_routes_to_individual(routes):
    """
    Кодирует список маршрутов в особь
    """
    individual = []

    for route in routes:
        for i, city in enumerate(route):
            if i == 0:  # Пропускаем начальный 'A'
                continue
            individual.append(vertex_to_code[city])

    return individual

def calculate_route_distance(route, vertices, routeskm):
    """
    Вычисляет расстояние маршрута
    """
    distance = 0
    for i in range(len(route) - 1):
        from_idx = vertices.index(route[i])
        to_idx = vertices.index(route[i+1])
        distance += routeskm[from_idx][to_idx]
    return distance

def check_route_constraints(route, data, available_cubics):
    """
    Проверяет ограничения по кубатуре для маршрута
    """
    volumes = calculate_route_volume_by_names(route)
    max_volume = max(volumes)
    min_max_cubic = min(data[city]['max_cubic'] for city in route)

    if max_volume > min_max_cubic:
        return False

    suitable_cubics = [c for c in available_cubics if c >= max_volume and c <= min_max_cubic]
    if not suitable_cubics:
        return False

    return True

def calculate_route_volume_by_names(route):
    """
    Вычисляет объем груза в каждой точке маршрута
    """
    volumes = []
    current_volume = data[route[0]]['volume_to_carry']

    for i in range(1, len(route)):
        if route[i] != 'END':
            current_volume += data[route[i]]['volume_to_route_point']

    volumes.append(current_volume)

    for i in range(1, len(route)-1):
        current_volume = current_volume - data[route[i]]['volume_to_route_point'] + data[route[i]]['volume_to_carry']
        volumes.append(current_volume)

    return volumes

# Модифицируем основной цикл генетического алгоритма
def main_with_intergenerational_optimization():
    population = toolbox.population(n=POPULATION_SIZE)
    hof = tools.HallOfFame(HOF_SIZE)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    logbook = tools.Logbook()

    for gen in range(MAX_GENERATIONS):
        # Выполняем стандартный шаг эволюции
        population = algorithms.varAnd(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION)

        # Вычисляем приспособленность
        fits = toolbox.map(toolbox.evaluate, population)
        for fit, ind in zip(fits, population):
            ind.fitness.values = fit

        # Обновляем зал славы
        hof.update(population)

        # Применяем межпоколенческую оптимизацию каждые 1000 поколений
        if gen % 1000 == 0:
            optimized_population = intergenerational_optimization(
                population[:10],  # Лучшие 10 особей
                coordinates,
                VERTICES,
                data,
                ROUTESKM,
                available_cubics
            )

            # Заменяем лучшие особи в популяции
            for i, ind in enumerate(optimized_population):
                population[i][:] = ind[:]
                population[i].fitness.values = ind.fitness.values

        # Селекция
        population = toolbox.select(population, k=len(population))

        # Сбор статистики
        record = stats.compile(population)
        logbook.record(gen=gen, **record)

    return population, logbook, hof

# Загрузка координат городов (предполагается, что у вас есть эта информация)
# coordinates = np.array([...])  # Ваши координаты городов

if __name__ == "__main__":
    # Загрузка данных и инициализация
    # ...

    # Запуск модифицированного алгоритма
    population, logbook, hof = main_with_intergenerational_optimization()

    # Применяем 2-opt к финальным решениям
    final_optimized_hof = apply_2opt_to_best_solutions(hof, data, VERTICES, ROUTESKM, available_cubics)

    # Выводим результаты
    print("Финальные оптимизированные решения:")
    for i, individual in enumerate(final_optimized_hof):
        print(f"Решение {i+1}:")
        # ... (код вывода маршрутов)

In [ ]:
dictionary = defaultdict(list)

In [ ]:
dictionary['MPID1111'] = [dot1111, dot2222]
dictionary['MPID2222'] = [dot3333, dot4444]

In [ ]:
!pip install ortools

import numpy as np
from scipy.spatial import distance
import itertools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def apply_2opt_with_geography(route, coordinates, vertices, routeskm):
    """
    Применяет алгоритм 2-opt с учетом географических данных
    """
    if len(route) <= 3:
        return route

    improved = True
    best_route = route[:]
    best_distance = calculate_route_distance_by_names(route, vertices, routeskm)

    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route) - 1):
                if j - i == 1:
                    continue

                new_route = best_route[:]
                new_route[i:j+1] = best_route[j:i-1:-1]

                new_distance = calculate_route_distance_by_names(new_route, vertices, routeskm)

                if new_distance < best_distance:
                    best_route = new_route
                    best_distance = new_distance
                    improved = True

    return best_route

def geographic_optimization(routes, coordinates, vertices, data, routeskm, available_cubics):
    """
    Географическая оптимизация маршрутов на основе координат городов
    """
    optimized_routes = []

    for route in routes:
        if len(route) <= 2:
            optimized_routes.append(route)
            continue

        # Применяем 2-opt для оптимизации порядка городов
        optimized_route = apply_2opt_with_geography(route, coordinates, vertices, routeskm)
        optimized_routes.append(optimized_route)

    # Перераспределяем города между маршрутами на основе географической близости
    optimized_routes = redistribute_cities_geographically(optimized_routes, coordinates, vertices, data, routeskm, available_cubics)

    return optimized_routes

def redistribute_cities_geographically(routes, coordinates, vertices, data, routeskm, available_cubics):
    """
    Перераспределяет города между маршрутами на основе географической близости
    """
    improved = True
    optimized_routes = [route[:] for route in routes]

    while improved:
        improved = False

        for i in range(len(optimized_routes)):
            if len(optimized_routes[i]) <= 2:
                continue

            for j in range(len(optimized_routes)):
                if i == j:
                    continue

                # Пытаемся переместить города из маршрута i в маршрут j
                for city_idx in range(1, len(optimized_routes[i]) - 1):
                    city = optimized_routes[i][city_idx]
                    if city in ['A', 'END']:
                        continue

                    # Проверяем, можно ли переместить город в другой маршрут
                    if can_move_city(city, optimized_routes[i], optimized_routes[j], data, available_cubics):
                        # Пробуем все возможные позиции в целевом маршруте
                        best_pos = None
                        best_improvement = 0

                        for pos in range(1, len(optimized_routes[j])):
                            # Создаем тестовые маршруты
                            test_route_i = optimized_routes[i][:]
                            test_route_i.remove(city)

                            test_route_j = optimized_routes[j][:]
                            test_route_j.insert(pos, city)

                            # Проверяем ограничения
                            if not check_route_constraints(test_route_j, data, available_cubics):
                                continue

                            # Вычисляем изменение расстояния
                            old_distance_i = calculate_route_distance_by_names(optimized_routes[i], vertices, routeskm)
                            old_distance_j = calculate_route_distance_by_names(optimized_routes[j], vertices, routeskm)
                            old_total = old_distance_i + old_distance_j

                            new_distance_i = calculate_route_distance_by_names(test_route_i, vertices, routeskm)
                            new_distance_j = calculate_route_distance_by_names(test_route_j, vertices, routeskm)
                            new_total = new_distance_i + new_distance_j

                            improvement = old_total - new_total

                            if improvement > best_improvement:
                                best_improvement = improvement
                                best_pos = pos

                        # Если нашли улучшение, применяем его
                        if best_improvement > 0:
                            optimized_routes[i].remove(city)
                            optimized_routes[j].insert(best_pos, city)
                            improved = True
                            break

                if improved:
                    break
            if improved:
                break

    return optimized_routes

def optimize_with_concorde(route, vertices, routeskm):
    """
    Оптимизирует маршрут с помощью алгоритма, подобного Concorde
    (упрощенная реализация для малого количества городов)
    """
    if len(route) <= 3 or len(route) > 12:
        return route

    # Извлекаем промежуточные города
    intermediate_cities = [city for city in route if city not in ['A', 'END']]

    # Создаем матрицу расстояний для промежуточных городов
    indices = [vertices.index(city) for city in intermediate_cities]
    dist_matrix = routeskm[np.ix_(indices, indices)]

    # Решаем задачу коммивояжера
    optimal_order = solve_tsp(dist_matrix)

    # Строим оптимизированный маршрут
    optimized_route = ['A'] + [intermediate_cities[i] for i in optimal_order] + ['END']

    return optimized_route

def solve_tsp(distance_matrix):
    """
    Решает задачу коммивояжера с помощью OR-Tools
    """
    # Создаем менеджер маршрутов
    manager = pywrapcp.RoutingIndexManager(
        len(distance_matrix),  # количество точек
        1,  # количество транспортных средств
        0   # начальная точка
    )

    # Создаем модель маршрутизации
    routing = pywrapcp.RoutingModel(manager)

    # Определяем функцию стоимости
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Настраиваем параметры поиска
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Решаем задачу
    solution = routing.SolveWithParameters(search_parameters)

    # Извлекаем решение
    if solution:
        index = routing.Start(0)
        route_nodes = []
        while not routing.IsEnd(index):
            route_nodes.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        return route_nodes

    return list(range(len(distance_matrix)))

def calculate_route_distance_by_names(route, vertices, routeskm):
    """
    Вычисляет расстояние маршрута по названиям городов
    """
    distance = 0
    for i in range(len(route) - 1):
        from_idx = vertices.index(route[i])
        to_idx = vertices.index(route[i+1])
        distance += routeskm[from_idx][to_idx]
    return distance

def can_move_city(city, source_route, target_route, data, available_cubics):
    """
    Проверяет, можно ли переместить город между маршрутами
    """
    # Проверяем, что город не находится в целевом маршруте
    if city in target_route:
        return False

    # Проверяем ограничения по кубатуре в целевом маршруте
    test_target_route = target_route + [city]
    if not check_route_constraints(test_target_route, data, available_cubics):
        return False

    # Проверяем ограничения по кубатуре в исходном маршруте
    test_source_route = [c for c in source_route if c != city]
    if len(test_source_route) > 2 and not check_route_constraints(test_source_route, data, available_cubics):
        return False

    return True

def check_route_constraints(route, data, available_cubics):
    """
    Проверяет ограничения по кубатуре для маршрута
    """
    volumes = calculate_route_volume_by_names(route)
    max_volume = max(volumes)
    min_max_cubic = min(data[city]['max_cubic'] for city in route)

    if max_volume > min_max_cubic:
        return False

    suitable_cubics = [c for c in available_cubics if c >= max_volume and c <= min_max_cubic]
    if not suitable_cubics:
        return False

    return True

def calculate_route_volume_by_names(route):
    """
    Вычисляет объем груза в каждой точке маршрута
    """
    volumes = []
    current_volume = data[route[0]]['volume_to_carry']

    for i in range(1, len(route)):
        if route[i] != 'END':
            current_volume += data[route[i]]['volume_to_route_point']

    volumes.append(current_volume)

    for i in range(1, len(route)-1):
        current_volume = current_volume - data[route[i]]['volume_to_route_point'] + data[route[i]]['volume_to_carry']
        volumes.append(current_volume)

    return volumes

# Основная функция оптимизации
def optimize_final_solutions(hof, coordinates, vertices, data, routeskm, available_cubics):
    """
    Применяет полную оптимизацию к лучшим решениям
    """
    optimized_solutions = []

    for individual in hof:
        # Декодируем особь в список маршрутов
        routes = []
        current_route = ['A']

        for gene in individual:
            city = code_to_vertex[gene]
            if city == 'END':
                current_route.append('END')
                routes.append(current_route)
                current_route = ['A']
            else:
                current_route.append(city)

        if len(current_route) > 1:
            current_route.append('END')
            routes.append(current_route)

        # Применяем географическую оптимизацию
        geo_optimized_routes = geographic_optimization(routes, coordinates, vertices, data, routeskm, available_cubics)

        # Применяем Concorde-оптимизацию к каждому маршруту
        final_routes = []
        for route in geo_optimized_routes:
            if len(route) <= 12:  # Concorde-оптимизация для маршрутов до 12 городов
                optimized_route = optimize_with_concorde(route, vertices, routeskm)
                final_routes.append(optimized_route)
            else:
                final_routes.append(route)

        # Создаем новую особь
        new_individual = []
        for route in final_routes:
            for i, city in enumerate(route):
                if i == 0:  # Пропускаем начальный 'A'
                    continue
                new_individual.append(vertex_to_code[city])

        new_ind = creator.Individual(new_individual)
        new_ind.fitness.values = toolbox.evaluate(new_ind)
        optimized_solutions.append(new_ind)

    return optimized_solutions

# Интеграция с основным алгоритмом
if __name__ == "__main__":
    # ... (предыдущий код загрузки данных и запуска GA)

    population, logbook, hof = main()

    # Применяем 2-opt к лучшим решениям
    optimized_hof = apply_2opt_to_best_solutions(hof, data, VERTICES, ROUTESKM, available_cubics)

    # Применяем финальную оптимизацию с использованием географических данных и Concorde
    final_optimized_hof = optimize_final_solutions(
        optimized_hof, coordinates, VERTICES, data, ROUTESKM, available_cubics
    )

    # Выводим результаты
    print("Финальные оптимизированные решения:")
    for i, individual in enumerate(final_optimized_hof):
        print(f"Решение {i+1}:")
        # ... (код вывода маршрутов)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatib

NameError: name 'main' is not defined